In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Dataset
from sklearn.datasets import load_digits

# PyTorch
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

trainset = torchvision.datasets.MNIST(root='~/data',
                                        train=True,
                                        download=True,
                                        transform=transform)

testset = torchvision.datasets.MNIST(root='~/data',
                                        train=False,
                                        download=True,
                                        transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 138218963.17it/s]

Extracting /root/data/MNIST/raw/train-images-idx3-ubyte.gz to /root/data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 118760484.14it/s]


Extracting /root/data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 47608465.83it/s]

Extracting /root/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 2269541.19it/s]


Extracting /root/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/data/MNIST/raw



In [4]:
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=2)

In [5]:
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=100,
                                         shuffle=False,
                                         num_workers=2)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # 28x28x32 -> 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3)  # 26x26x64 -> 24x24x64
        self.pool = nn.MaxPool2d(2, 2)  # 24x24x64 -> 12x12x64
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.dropout2 = nn.Dropout2d()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [9]:
torch.cuda.is_available()

True

In [11]:
device = torch.device("cuda:0")

In [12]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [16]:
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[{epoch + 1}, {i+1}] loss: {running_loss / 100:.2}')
            running_loss = 0.0

print('Finished Training')

[1, 100] loss: 2.2
[1, 200] loss: 1.6
[1, 300] loss: 0.82
[1, 400] loss: 0.62
[1, 500] loss: 0.51
[1, 600] loss: 0.48
[2, 100] loss: 0.44
[2, 200] loss: 0.41
[2, 300] loss: 0.38
[2, 400] loss: 0.37
[2, 500] loss: 0.35
[2, 600] loss: 0.34
[3, 100] loss: 0.33
[3, 200] loss: 0.31
[3, 300] loss: 0.29
[3, 400] loss: 0.29
[3, 500] loss: 0.28
[3, 600] loss: 0.26
[4, 100] loss: 0.26
[4, 200] loss: 0.24
[4, 300] loss: 0.25
[4, 400] loss: 0.24
[4, 500] loss: 0.24
[4, 600] loss: 0.24
[5, 100] loss: 0.22
[5, 200] loss: 0.21
[5, 300] loss: 0.22
[5, 400] loss: 0.22
[5, 500] loss: 0.21
[5, 600] loss: 0.21
Finished Training
